# Intro 

> This tutorial is a quick introduction to [Folium](https://folium.readthedocs.io/), which you will be using in _HW3_ to build maps. 

> **NOTE** This notebook is not a thorough reference or guide to `folium`. For this please check out [`Folium`'s documentation](https://folium.readthedocs.io/en/latest/quickstart.html).

---

# Installation

We recommend using the latest version : `0.5.0`.

`Folium` is a regular python package, which can be installed through several means :

### 1. Conda
```
conda install folium
```

### 2. pip

```
pip install -U folium
``` 

By default, the `pip` command is linked to the local `python` distribution on. To use with your notebook, make sure you use the pip bundled with `anaconda`. On mac for example, this is usually : 
```
~/anaconda/bin/pip install -U folium
```

### 3. Direct download

The package is available [directly from pypi](https://pypi.python.org/pypi/folium)

---

# Getting Started

Start by importing some standard libraries for processing.

In [ ]:
import os
import pandas as pd
import json

And of course, `folium`.

In [ ]:
import folium

Make sure you are you using the latest version of folium `0.5.0`.

In [ ]:
folium.__version__ == '0.5.0'

# The basics

## Loading a map

Let's start by the first step of this visualisation exercise : the map. This is vers simple thanks to `folium`. Just start by defining some base coordinates ([`lat`, `lng`] format).

Let's start from home :

In [ ]:
lausanne_coord = [46.519962, 6.633597]

Now we just pass these coordinates to the basic `Map` object to construct it :

In [ ]:
m = folium.Map(location=lausanne_coord)
m

> **NOTE** Though the map loads nicely in your notebook environment, note that it will not render on github ! We will discuss how to save your notebook to HTML later on. You can also decide to use an online viewer such as [this one](https://nbviewer.jupyter.org/).

Folium maps use standard mapping technology, composing tiles to build the maps you see. Several tile sources are provided for different uses or styles. Folium supports passing any `leaflet.js` compatible custom tileset.

In [ ]:
folium.Map(location=lausanne_coord,
           tiles='Stamen Toner',
           zoom_start=13)

## Markers

Another important feature of `folium` is how easy it is to add interactivity. Let's add a simple marker to our Lausanne map to point at **EPFL**.

We start by defining the location of the marker, with the same format as before :

In [ ]:
epfl_location = [46.519962, 6.633597]

We create a marker, giving it a popup string (e.g. "EPFL"). We can then place the marker on our map by using the `add_to` function :

In [ ]:
folium.Marker(epfl_location, popup="EPFL").add_to(m)

Show the map again, with the marker on it ! You can click on it to reveal the popup.

In [ ]:
m

# Overlays

The other important part you will need for the homework are overlays. These allow you to draw shapes on a map, such as boundaries, borders, roads, ...

They come in the form of `json` files (formally `.topojson` or `.geojson`), and can be loaded onto `folium` maps very easily.

Let's load a `.geojson` file that shows the borders of states in the US.

In [ ]:
state_geo_path = r'us-states.json'
geo_json_data = json.load(open(state_geo_path))

Now let's build a blank map (using the same method as in the introduction) of the US :

In [ ]:
m_usa = folium.Map([43,-100], tiles='cartodbpositron', zoom_start=4)
m_usa

Now we can overlay the state boundaries on this map :

In [ ]:
folium.GeoJson(geo_json_data).add_to(m_usa)
m_usa

This is cool but we want to be able to display some data on this map. Using the example from the slides, let's build a simple map that will show who won the 2016 general election in each state.

This is data is made available in `US_Election_2016_binary.csv`.

In [ ]:
state_data_file = r'US_Election_2016_binary.csv'
state_data = pd.read_csv(state_data_file)

The data is formatted (arbitrarily) as :
* `1` if Trump won
* `2` if Clinton won

In [ ]:
state_data.head()

`folium` allows us to specify a custom coloring function, that will style each polygon in a particular way, dependant on the data. Let's make a function that will color a state **blue** if Clinton won, **red** if Trump won.

In [ ]:
def us_election_colors(state):
    """Maps Clinton wins to blue and Trump wins to red."""
    clinton_win = state_data.loc[state_data['State'] == state, 'Winner'].values[0] > 1
    if clinton_win:
        return '#0000ff'
    else:
        return '#ff0000'

Okay, now we're ready to see who won :

In [ ]:
results_map = folium.Map([43,-100], tiles='cartodbpositron', zoom_start=4)
folium.GeoJson(
    geo_json_data,
    style_function=lambda feature: {
        'fillColor': us_election_colors(feature['id']),
        'color' : 'black',
        'weight' : 2,
        'dashArray' : '5, 5'
        }
    ).add_to(results_map)

results_map

This is an example of a [Choropleth map](https://en.wikipedia.org/wiki/Choropleth_map).

>  thematic map in which areas are shaded or patterned in proportion to the measurement of the statistical variable being displayed on the map, such as population density or per-capita income. Choropleth maps can also be used to display nominal data such as country names on a world map or most popular car model per region.

A very common type of map visualization, which you will be using in the homework. Luckily, it is so common that `folium` has a pre-defined class that you can use to build them. To see how it works, let's load some more detailed data about the elections : instead of just who won, let's load the percentage of votes Trump (the winner, *and bigly, everybody says so*) got in each state. 

In [ ]:
state_winner_path = r'US_Election_2016.csv'
state_winner_data = pd.read_csv(state_winner_path)

In [ ]:
state_winner_data.head()

Folium will take care of most of the scaling for you ! You can just say what data you want to be plotted where (make sure the keys for the data and the keys in the geojson match), and Folium will do the rest. Think about the colors you want to use, add a legend and you're good to go.

In [ ]:
us_map = folium.Map(location=[48, -102], zoom_start=3)
us_map.choropleth(geo_data=geo_json_data, data=state_winner_data,
             columns=['State', 'Percentage'],
             key_on='feature.id',
             fill_color='BuPu', fill_opacity=0.7, line_opacity=0.2,
             legend_name='Percentage for winner of general election (%)')
us_map

This offers a more detailed view of what actually happened ! The colors could use some refinement though. Try to think of what colors would be most expressive in this case. You can find a detailed list [here](https://github.com/python-visualization/folium/blob/v0.2.0/folium/utilities.py#L104), or use the resources at your disposal to select an appropriate colorscheme. Behind the scenes, this part uses the [colormaps](https://nbviewer.jupyter.org/github/python-visualization/folium/blob/v0.2.0/examples/Colormaps.ipynb) we saw earlier, so it is an option to define your own.

# Saving your plots

You now have the basic tools at your disposal to build great visualizations, but what do you do with them afterwards ? As you may have noticed, `folium`'s documentation is a jupyter notebook ! This is one option to present mixed media. Nevertheless, the library provides rendering capabilities that are very useful and make your work exportable anywhere there is a browser :

In [ ]:
us_map.save('US_Election_2016.html')
results_map.save('US_Election_2016_binary.html')

You can now access your maps directly [in your browser](US_Election_2016.html). They can also be included on any other webpage (for example you can host a website on [Github Pages](https://pages.github.com/) for free !)